In [34]:
from pyqubo import Array, Constraint, Placeholder
import time
import numpy as np
import math

In [99]:
x = []
y = []
# with open('../data/data.txt','r') as file:
with open('../coeff-generator/test_data.txt','r') as file:
    for line in file:
        cord = line.split()[:2]
        tx = float(cord[0])
        ty = float(cord[1])
        x.append(tx)
        y.append(ty)

In [100]:
ncity = 3
city_distance = np.zeros((ncity, ncity), dtype=np.float64)
for i in range(0, ncity):
    for j in range(i + 1, ncity):
        city_distance[i][j] = math.sqrt((x[i] - x[j]) ** 2 + (y[i] - y[j]) ** 2)
        city_distance[j][i] = city_distance[i][j]

In [106]:
n_city = ncity

t0 = time.time()
x = Array.create('c', (n_city, n_city), 'BINARY')

# Constraint not to visit more than two cities at the same time.
time_const = 0.0
for i in range(n_city):
    # If you wrap the hamiltonian by Const(...), this part is recognized as constraint
    time_const += Constraint((sum(x[i, j] for j in range(n_city)) - 1)**2, label="time{}".format(i))

# Constraint not to visit the same city more than twice.
city_const = 0.0
for j in range(n_city):
    city_const += Constraint((sum(x[i, j] for i in range(n_city)) - 1)**2, label="city{}".format(j))

# distance of route
distance = 0.0
for k in range(n_city):
    for i in range(n_city):
        for j in range(n_city):
            # we set the constant distance
            d_ij = city_distance[i][j]
            distance += 0.5 * d_ij * x[k, i] * (x[(k+1)%n_city, j] + x[(k-1)%n_city, j])

# Construct hamiltonian
A = Placeholder("A")
H = distance + A * (time_const + city_const)

# Compile model
t1 = time.time()
model = H.compile()
qubo, offset = model.to_qubo(index_label=True, feed_dict={"A": 2.0})
linear,quadratic,offset = model.to_ising(index_label=True, feed_dict={"A": 1.0})
t2 = time.time()

In [107]:
linear = dict(sorted(linear.items(), key=lambda x: x[0]))
with open('linear.txt','w') as file:
    for key,value in linear.items():
        file.write(str(key)+" "+str(value)+"\n")
linear

{0: 3.1213203435596424,
 1: 3.121320343559643,
 2: 3.121320343559643,
 3: 3.121320343559643,
 4: 3.121320343559643,
 5: 3.1213203435596424,
 6: 2.4142135623730954,
 7: 2.4142135623730954,
 8: 2.4142135623730954}

In [108]:
a = np.zeros((ncity**2, ncity**2))
for key, valuie in quadratic.items():
    a[key] = value
a

array([[0.        , 2.41421356, 2.41421356, 2.41421356, 2.41421356,
        2.41421356, 2.41421356, 2.41421356, 2.41421356],
       [0.        , 0.        , 2.41421356, 2.41421356, 2.41421356,
        2.41421356, 2.41421356, 2.41421356, 2.41421356],
       [0.        , 0.        , 0.        , 2.41421356, 2.41421356,
        2.41421356, 2.41421356, 2.41421356, 2.41421356],
       [0.        , 0.        , 0.        , 0.        , 2.41421356,
        2.41421356, 2.41421356, 2.41421356, 2.41421356],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        2.41421356, 2.41421356, 2.41421356, 2.41421356],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 2.41421356, 2.41421356, 2.41421356],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 2.41421356, 2.41421356],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 2.41421356],


In [109]:
quadratic = dict(sorted(quadratic.items(), key=lambda x: x[0]))
with open('quadratic.txt','w') as file:
    for key,value in quadratic.items():
        file.write(str(key[0])+" "+str(key[1])+" "+str(value)+"\n")

In [110]:
offset

14.485281374238568